In [1]:
import subprocess as sp

python train.py --config=yolact_pp_101_ordered_rc_ucb_gait_config --resume=weights/yolact_plus_base_54_800000.pth --start_iter=0 --batch_size=5 --lr=0.0001 --momentum 0.6 --gamma 0.1 --save_interval 1000 --validation_size 1000 --keep_latest_interval 1000 --validation_iter 1000 --only_last_layer 2>&1 | tee "yolact_pp_ordered_101_rc_ucb_gait.txt"

In [5]:
def train_yolact():
    FFMPEG_BIN = "python"
    command = [ FFMPEG_BIN,'/home/josmar/proyectos/yolact/train.py',
                '--config', 'yolact_pp_101_ordered_rc_ucb_gait_config', 
                '--resume', '/home/josmar/proyectos/yolact/weights/yolact_plus_base_54_800000.pth',
                '--start_iter', str(0),
                '--batch_size', str(5),
                '--lr', str(0.0001),
                '--momentum', str(0.6),
                '--gamma', str(0.1),
                '--save_interval', str(1000),
                '--validation_size', str(1000),
                '--keep_latest_interval', str(1000),
                '--validation_iter', str(1000),
                '--only_last_layer']

    print("Starting training script ...")
    sp.run(command)
    print("Finished training script ...")

In [6]:
train_yolact()

Starting training script ...


KeyboardInterrupt: 

## Trying other method

In [1]:
import sys
sys.path.append('/home/josmar/proyectos/yolact')

from data import *
from utils.augmentations import SSDAugmentation, BaseTransform, FastBaseTransform, Resize
from utils.functions import MovingAverage, SavePath
from utils.logger import Log
from utils import timer
from layers.modules import MultiBoxLoss
from layers.output_utils import postprocess, undo_image_transformation #To vuisualize
from yolact import Yolact
import os
import sys
import time
import math, random
from pathlib import Path
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
import datetime
import wandb
import random

# Oof
import eval as eval_script
import matplotlib.pyplot as plt

In [2]:
def predict_images (my_args, n_imgs):
    global args
    args = my_args
    if args.config is not None:
        set_cfg(args.config)

    if args.trained_model == 'interrupt':
        args.trained_model = SavePath.get_interrupt('weights/')
    elif args.trained_model == 'latest':
        args.trained_model = SavePath.get_latest('weights/', cfg.name)

    if args.config is None:
        model_path = SavePath.from_str(args.trained_model)
        # TODO: Bad practice? Probably want to do a name lookup instead.
        args.config = model_path.model_name + '_config'
        print('Config not specified. Parsed %s from the file name.\n' % args.config)
        set_cfg(args.config)

    # if args.detect:
    #     cfg.eval_mask_branch = False

    # if args.dataset is not None:
    #     set_dataset(args.dataset)

    with torch.no_grad():
        # if not os.path.exists('results'):
        #     os.makedirs('results')

        cudnn.fastest = True
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        
        
        print('Loading model...', end='')
        net = Yolact()
        net.load_weights(args.trained_model)
        net.eval()
        print(' Done.')
        
        net = net.cuda()

        return evaluate(net, n_imgs)

def evaluate(net:Yolact, train_mode=False, n_imgs = 3):
    net.detect.use_fast_nms = True
    net.detect.use_cross_class_nms = False
    cfg.mask_proto_debug = False
    
    top_k = 15
    th = 0.4
    mask_alpha = 1

    test_imgs = os.listdir(cfg.dataset.test_images)
    
    class_labels = {
        1: "person 1",
        2: "person 2",
        3: "person 3"
        }

    wandb_imgs = []
    for img_path in random.sample(test_imgs, n_imgs):
        img_path = os.path.join(cfg.dataset.test_images, img_path)
        print(img_path)

        # Find predictions for the selected path
        preds = evalimage(net=net, path = img_path)
        
        # Load the image and change it to RGB
        im = cv2.imread(img_path)
        rgb_im = im[:, :, ::-1]

        # Getting image size
        h,w,_ = rgb_im.shape
        
        #Process predictions
        save = cfg.rescore_bbox
        cfg.rescore_bbox = True
        t = postprocess(preds, w, h, visualize_lincomb = False,
                                            crop_masks        = True,
                                            score_threshold   = th)
        cfg.rescore_bbox = save

        idx = t[1].argsort(0, descending=True)[:15]
        #Getting masks,classes scores and boxes from the prediction
        masks = t[3][idx]
        classes, scores, boxes = [x[idx].cpu().numpy() for x in t[:3]]
        
        # Formatting results into wandb
        bin_mask= np.zeros((h, w))
        box_data = []
        
        class2name = {0:"person"}
        for j in range(classes.shape[0]):
            if scores[j] > th:
                # masks
                sil = masks[j].byte().cpu().numpy()
                av_mask = bin_mask == 0 # Gets the pixels that are not used 
                corrected_sil = np.bitwise_and(av_mask, sil) #Gets the silhouette cropping the overlapped parts
                bin_mask+= corrected_sil * (j+1)

                # scores 
                acc = scores[j]
                acc = round(float(acc), 2)

                #boxes
                x_min, y_min, x_max, y_max = boxes[j]
                x_min, x_max = round(x_min/w, 2), round(x_max/w, 2)
                y_min, y_max = round(y_min/h, 2), round(y_max/h, 2)
                bbox_dict = {"position": {
                                "minX": x_min,
                                "maxX": x_max,
                                "minY": y_min,
                                "maxY": y_max},
                            "class_id" : j+1,
                            "box_caption": "person:{}".format(acc),
                            "scores" : {
                                "acc": acc},
                            }
                box_data.append(bbox_dict)

                #class_labels
                # class_str = class2name[classes[j]]+ " " + str(j+1)
                # class_labels[j+1] = class_str
        wandb_img = wandb.Image(rgb_im, 
        masks={"prediction" : {"mask_data" : bin_mask, "class_labels" : class_labels}},
        boxes={"prediction" : {"box_data" : box_data, "class_labels" : class_labels}})
        wandb_imgs.append(wandb_img)
        # plt.imshow(bin_mask)
        # plt.show()
        # plt.imshow(rgb_im)
        # plt.show()
        # print(box_data)
    return wandb_imgs
        

def evalimage(net:Yolact, path:str):
    input_img = cv2.imread(path)
    frame = torch.from_numpy(input_img).cuda().float()
    batch = FastBaseTransform()(frame.unsqueeze(0))
    preds = net(batch)

    return preds

In [3]:
class Args:
    def __init__(self, config):
        self.batch_size = 5
        self.resume = "yolact_data/weights/yolact_plus_base_54_800000.pth"
        self.start_iter = 0
        self.num_workers = 4
        self.cuda = True
        self.lr = config.lr
        self.momentum = config.momentum #0.6
        self.decay = config.decay
        self.gamma = None
        self.save_folder = 'yolact_data/train_weights/'
        self.log_folder = 'yolact_data/logs/'
        self.config = "yolact_pp_101_ordered_rc_ucb_gait_config"
        self.save_interval = 1000 #10000
        self.validation_size = 1000 #5000
        self.validation_iter = 1000 #10000
        self.keep_latest = False
        self.keep_latest_interval = 1000
        self.dataset = None
        self.log = True
        self.log_gpu = False
        self.interrupt = True
        self.batch_alloc = None
        self.autoscale = True
        self.eval_only_person = False
        self.only_last_layer = config.only_last_layer
        self.compute_val_loss = False
        self.max_iter = config.max_iter
        self.trained_model = "latest"

In [4]:
class my_class:
    def __init__(self):

        self.a=1
        self.b=2
        self.c=3
        self.d=4

In [13]:
c = my_class()
print(vars(c))

{'a': 1, 'b': 2, 'c': 3, 'd': 4}


In [4]:
my_config = {
    "batch_size" : 5,
    "resume" : "yolact_data/weights/yolact_plus_base_54_800000.pth",
    "start_iter" : 0,
    "num_workers" : 4,
    "cuda" : True,
    "lr" : 0.0001,
    "momentum" : 0.9, #0.6
    "decay" : None,
    "gamma" : 0.1,
    "save_folder" : 'weights/',
    "log_folder" : 'yolact_data/logs/',
    "config" : "yolact_pp_101_ordered_rc_ucb_gait_config",
    "save_interval" : 1000, #10000
    "validation_size" : 1000, #5000
    "validation_iter" : 1000, #10000
    "keep_latest" : False,
    "keep_latest_interval" : 1000,
    "dataset" : None,
    "log" : True,
    "log_gpu" : False,
    "interrupt" : True,
    "batch_alloc" : None,
    "autoscale" : True,
    "eval_only_person" : False,
    "only_last_layer" : True,
    "compute_val_loss" : False,
    "max_iter" : 2000}

In [178]:
global args
    
wandb.init(project="instance_segmentation_train")
config = wandb.config
config.update(my_config) # Used for manual parameters
args = Args(config)
args.lr = config.lr
args.momentum = config.momentum
args.decay = config.decay
# set_args()
os.environ['WANDB_NOTEBOOK_NAME'] = 'yolact_loop.py'
config.framework = "pytorch"
predicted_images = predict_images(args, n_imgs=3)
wandb.log({"predictions" : predicted_images})
wandb.run.finish()

Loading model... Done.
/home/josmar/proyectos/codes/datasets/ucb_gait_cropped/images/crop_024-077.jpg
/home/josmar/proyectos/codes/datasets/ucb_gait_cropped/images/crop_013-230.jpg
/home/josmar/proyectos/codes/datasets/ucb_gait_cropped/images/crop_023-162.jpg


_step,0
_runtime,2
_timestamp,1611464813


_step,▁
_runtime,▁
_timestamp,▁


In [39]:
my_preds[0].keys()

dict_keys(['detection', 'net'])

In [14]:
my_preds[0]['detection']['proto']

NameError: name 'a' is not defined

In [129]:
path = "/home/josmar/proyectos/codes/04_casia_tests/input/people walking.jpg"
input_img = cv2.imread(path)
rgb_im = input_img[:, :, ::-1]
frame = torch.from_numpy(input_img).cuda().float()

In [102]:
h,w,_ = input_img.shape
top_k = 15
th = 0.4
mask_alpha = 1

save = cfg.rescore_bbox
cfg.rescore_bbox = True
t = postprocess(my_preds, w, h, visualize_lincomb = False,
                                        crop_masks        = True,
                                        score_threshold   = th)
cfg.rescore_bbox = save
        

In [103]:
idx = t[1].argsort(0, descending=True)[:15]
masks = t[3][idx]
classes, scores, boxes = [x[idx].cpu().numpy() for x in t[:3]]

In [104]:
num_dets_to_consider = min(top_k, classes.shape[0])
for j in range(num_dets_to_consider):
    if scores[j] < th:
        num_dets_to_consider = j
        break

In [131]:
bin_mask= np.zeros((h, w))
box_data = []
class_labels = {}
class2name = {0:"person"}
for j in range(classes.shape[0]):
    if scores[j] > th:
        # masks
        sil = masks[j].byte().cpu().numpy()
        av_mask = bin_mask == 0 # Gets the pixels that are not used 
        corrected_sil = np.bitwise_and(av_mask, sil) #Gets the silhouette cropping the overlapped parts
        bin_mask+= corrected_sil * (j+1)

        # scores 
        acc = scores[j]
        acc = round(float(acc), 2)

        #boxes
        x_min, y_min, x_max, y_max = boxes[j]
        x_min, x_max = round(x_min/w, 2), round(x_max/w, 2)
        y_min, y_max = round(y_min/h, 2), round(y_max/h, 2)
        bbox_dict = {"position": {
                        "minX": x_min,
                        "maxX": x_max,
                        "minY": y_min,
                        "maxY": y_max},
                    "class_id" : j+1,
                    "box_caption": "person:{}".format(acc),
                    "scores" : {
                        "acc": acc},
                    }
        box_data.append(bbox_dict)

        #class_labels
        class_str = class2name[classes[j]]+ " " + str(j+1)
        class_labels[j+1] = class_str
wandb_img = wandb.Image(rgb_im, 
masks={"prediction" : {"mask_data" : bin_mask, "class_labels" : class_labels}},
boxes={"prediction" : {"box_data" : box_data, "class_labels" : class_labels}})

In [123]:
print(box_data)
print(class_labels)

[{'position': {'minX': 0.39, 'maxX': 0.55, 'minY': 0.27, 'maxY': 0.9}, 'class_id': 1, 'box_caption': 'person:0.94', 'scores': {'acc': 0.94}}, {'position': {'minX': 0.54, 'maxX': 0.7, 'minY': 0.29, 'maxY': 0.9}, 'class_id': 2, 'box_caption': 'person:0.91', 'scores': {'acc': 0.91}}]
{1: 'person 1', 2: 'person 2'}
person 2


In [132]:
wandb_img

In [13]:
masks_color = masks.repeat(1, 1, 1, 3) * mask_alpha
inv_alph_masks = masks * (-mask_alpha) + 1
        
# I did the math for this on pen and paper. This whole block should be equivalent to:
#    for j in range(num_dets_to_consider):
#        img_gpu = img_gpu * inv_alph_masks[j] + masks_color[j]
masks_color_summand = masks_color[0]
if num_dets_to_consider > 1:
    inv_alph_cumul = inv_alph_masks[:(num_dets_to_consider-1)].cumprod(dim=0)
    masks_color_cumul = masks_color[1:] * inv_alph_cumul
    masks_color_summand += masks_color_cumul.sum(dim=0)

RuntimeError: The size of tensor a (2640) must match the size of tensor b (880) at non-singleton dimension 3

##